In [1]:
from osgeo import gdal, gdal_array, ogr
import numpy as np
import os

#Chemin vers l'image d'entrée
band_path = "../../data/images"
band_template = os.path.join(band_path, "SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif")

#Chemin pour le fichier de masque en sortie
output_mask_path = "../results/data/img_pretraitees/masque_foret.tif"

#Chemin pour le fichier vecteur BD Forêt
vector_foret_path = "../../data/project"
form_vege = os.path.join(vector_foret_path,"FORMATION_VEGETALE.shp")

#Charger l'image raster
dataset = gdal.Open(band_template)
band = dataset.GetRasterBand(1)  # Charger une bande spécifique (par exemple, la bande 1)

#Charger le vecteur (BD Forêt avec les colonnes attributaires)
vector_ds = ogr.Open(form_vege)
layer = vector_ds.GetLayer()

#Définir les valeurs de TFV à exclure
excluded_tfv = [
    'FO0', 'FO1', 'FO2', 'FO3', 'LA4', 'LA6'
]

#Créer un raster temporaire pour les zones à inclure
raster_ds = gdal.Open(band_template)
raster_geotransform = raster_ds.GetGeoTransform()
raster_projection = raster_ds.GetProjection()
cols = raster_ds.RasterXSize
rows = raster_ds.RasterYSize

#Créer un raster de sortie (1 = à classer, 0 = à exclure)
driver = gdal.GetDriverByName("GTiff")
mask_ds = driver.Create(output_mask_path, cols, rows, 1, gdal.GDT_Byte)
mask_ds.SetGeoTransform(raster_geotransform)
mask_ds.SetProjection(raster_projection)

#Rasteriser les polygones à inclure
mask_band = mask_ds.GetRasterBand(1)
mask_band.Fill(0)  # Par défaut, exclure tout
mask_band.SetNoDataValue(0)

#Appliquer un filtre pour inclure uniquement les polygones avec des TFV non exclus
excluded_tfv_str = "', '".join(excluded_tfv)
layer.SetAttributeFilter(f"CODE_TFV NOT IN ('{excluded_tfv_str}')")

# Rasteriser les polygones filtrés dans le masque
gdal.RasterizeLayer(mask_ds, [1], layer, burn_values=[1])  # 1 = à classer

#Nettoyer les objets
mask_band.FlushCache()
mask_ds = None
vector_ds = None
raster_ds = None

print(f"Mask successfully created at {output_mask_path}")

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Mask successfully created at ../results/data/img_pretraitees/masque_foret.tif
